<a href="https://pymc-devs.github.io/pymc/tutorial.html">PyMC Tutorial</a>
<h3>The Model:</h3>

In [23]:
from pymc import DiscreteUniform, Exponential, deterministic, Poisson, Uniform
import numpy as np

disasters_array = \
        np.array([ 4, 5, 4, 0, 1, 4, 3, 4, 0, 6, 3, 3, 4, 0, 2, 6,
                   3, 3, 5, 4, 5, 3, 1, 4, 4, 1, 5, 5, 3, 4, 2, 5,
                   2, 2, 3, 4, 2, 1, 3, 2, 2, 1, 1, 1, 1, 3, 0, 0,
                   1, 0, 1, 1, 0, 0, 3, 1, 0, 3, 2, 2, 0, 1, 1, 1,
                   0, 1, 0, 1, 0, 0, 0, 2, 1, 0, 0, 0, 1, 1, 0, 2,
                   3, 3, 1, 1, 2, 1, 1, 1, 1, 2, 4, 2, 0, 0, 1, 4,
                   0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1])

# switchpoint, all values equally likely a priori
switchpoint = DiscreteUniform('switchpoint', lower=0, upper=110, doc='Switchpoint[year]')

early_mean = Exponential('early_mean', beta=1.)
late_mean = Exponential('late_mean', beta=1.)

"""In PyMC terminology, the parameter rate is 'deterministic', meaning it's determined explicitly
by observations of its parents. (It's still a random variable)"""
@deterministic(plot=False)
def rate(s=switchpoint, e=early_mean, l=late_mean):
    # concatenate poisson means
    out = np.empty(len(disasters_array)) # array of size 1 x len(disasters_array) with uninitialized values
    out[:s] = e
    out[s:] = l
    return out                           # looks like [e, e, ..., e, l, l,...,l]

"""Finally, we model the observed random variable, disasters. We have to assign it a probability distribution
because we want to use Bayes' Theorem"""
disasters = Poisson('disasters', mu=rate, value=disasters_array, observed=True) # note the declaration of observed

In [24]:
import disaster_model
disaster_model.switchpoint.parents

{'lower': 0, 'upper': 110}

In [25]:
disaster_model.disasters.parents

{'mu': <pymc.PyMCObjects.Deterministic 'rate' at 0x1042cb090>}

In [26]:
disaster_model.rate.children

{<pymc.distributions.Poisson 'disasters' at 0x1042cb150>}

Notice that parents are dictionaries while children are sets

In [27]:
disaster_model.disasters.value # we set this value

array([4, 5, 4, 0, 1, 4, 3, 4, 0, 6, 3, 3, 4, 0, 2, 6, 3, 3, 5, 4, 5, 3, 1,
       4, 4, 1, 5, 5, 3, 4, 2, 5, 2, 2, 3, 4, 2, 1, 3, 2, 2, 1, 1, 1, 1, 3,
       0, 0, 1, 0, 1, 1, 0, 0, 3, 1, 0, 3, 2, 2, 0, 1, 1, 1, 0, 1, 0, 1, 0,
       0, 0, 2, 1, 0, 0, 0, 1, 1, 0, 2, 3, 3, 1, 1, 2, 1, 1, 1, 1, 2, 4, 2,
       0, 0, 1, 4, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1])

In [28]:
disaster_model.switchpoint.value # the following values were randomly assigned

array(66)

In [29]:
disaster_model.early_mean.value 

array(2.357683692267941)

In [30]:
disaster_model.late_mean.value

array(0.7417309029132656)

In [31]:
disaster_model.rate.value

array([ 2.35768369,  2.35768369,  2.35768369,  2.35768369,  2.35768369,
        2.35768369,  2.35768369,  2.35768369,  2.35768369,  2.35768369,
        2.35768369,  2.35768369,  2.35768369,  2.35768369,  2.35768369,
        2.35768369,  2.35768369,  2.35768369,  2.35768369,  2.35768369,
        2.35768369,  2.35768369,  2.35768369,  2.35768369,  2.35768369,
        2.35768369,  2.35768369,  2.35768369,  2.35768369,  2.35768369,
        2.35768369,  2.35768369,  2.35768369,  2.35768369,  2.35768369,
        2.35768369,  2.35768369,  2.35768369,  2.35768369,  2.35768369,
        2.35768369,  2.35768369,  2.35768369,  2.35768369,  2.35768369,
        2.35768369,  2.35768369,  2.35768369,  2.35768369,  2.35768369,
        2.35768369,  2.35768369,  2.35768369,  2.35768369,  2.35768369,
        2.35768369,  2.35768369,  2.35768369,  2.35768369,  2.35768369,
        2.35768369,  2.35768369,  2.35768369,  2.35768369,  2.35768369,
        2.35768369,  0.7417309 ,  0.7417309 ,  0.7417309 ,  0.74

<p><tt>logp</tt> returns the log of the probability density of a stochastic variable at their current values given the values
of their parents. We can think of the -log probability as a scale of rarity, the larger the more rare.</p>

In [32]:
disaster_model.switchpoint.logp # log of uniform probability evaluated at switchpoint.value

-4.709530201312334

In [33]:
disaster_model.disasters.logp # sum of logs of joint density for each element of disasters 
                              # given parents' (the parameters') values

-185.07848122973684

In [34]:
disaster_model.early_mean.logp # e~exp(1) so log(1*exp(-1*e)) = -e

-2.357683692267941

In [35]:
disaster_model.late_mean.logp

-0.7417309029132656

In [36]:
from math import exp

exp(disaster_model.switchpoint.logp) # because this came from a uniform(0,110) distribution it is == 1/111

0.00900900900900901

In [54]:
"""What logp is really doing for vector valued objects"""
from scipy import stats


arr = np.array([np.log(stats.poisson.pmf(disaster_model.disasters.value[i], mu=disaster_model.rate.value[i])) 
                for i in range(disaster_model.disasters.value.shape[0])])
np.sum(arr)

-185.07848122973684

In [55]:
disaster_model.rate.parents.value # although these values are stochastic, 

{'e': array(2.357683692267941), 'l': array(0.7417309029132656), 's': array(66)}

In [56]:
from pymc import MCMC

#Markov Chain Monte Carlo Sampling
M = MCMC(disaster_model)

M.sample(iter=10000, burn=1000, thin=10)

 [-----------------100%-----------------] 10000 of 10000 complete in 2.1 sec

<p>M.sample creates a series of retained samples of (<tt>s, e, l</tt>) from the posterior distribution of the model. <br>
The MCMC joint distribution for <strong>P</strong>(<tt>s, e, l|D)</tt> converges to the true distribution, so we <tt>burn</tt> the <br>
first 1000 values. Because the values are generated by a autoregressive stochastic process close by values <br>
of the samples will be strongly autocorrelated so we take only every 10 values (<tt>thin=10</tt>)</p>